In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [1]:
url = "https://www.millenaire3.com/ressources/2023/changer-les-comportements-revue-et-enseignements-des-actions-metropolitaines"
url2 = "https://www.millenaire3.com/ressources/2023/agir-sur-les-modes-de-vie-une-nouvelle-grille-d-analyse"
url3 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire"
url4 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/institut-transitions-notre-but-c-est-de-former-ceux-qui-peuvent-agir-des-aujourd-hui"
url5 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/les-alchimistes-lyon-alimentation-gaspillage-compostage-aucun-de-ces-sujets-ne-laisse-indifferent"
url6 = "https://www.millenaire3.com/ressources/2023/crise-ecologique-le-metabolisme-de-l-economie-au-caeur-des-enjeux"
urls = [url, url2, url3, url4, url5, url6]

In [6]:
def scrap_page(url):
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    print(url)

    #TITRE
    titre = soup.find("h1", {"class": "uppercase h1-special"}).contents[0]
    titre = titre.replace("\n", "").replace("\t", "")
    print(titre)

    #TYPE
    type_liste = soup.find_all("p", {"class" : re.compile("^over-under")})
    for typ in type_liste:
        if re.search("Étude|Dossier|Interview", typ.text) == None:
            pass
        else:
            if re.search("Interview", typ.text) == None:
                type = typ.text.replace("\n", "").replace("\t", "").replace(" ", "")
            else:
                type = "Interview"
                itw_de = typ.text.replace("\n", "").replace("\t", "")
    print(type)
    
    #RESUME
    if type == "Dossier":
        resume = soup.find("div", {"class" : "eztext-field"})
    elif type == "Interview":
        resume = soup.find("div", {"class" : "ezrichtext-field"})
    else: 
        resume = soup.find("div", {"class" : "eztext-field"})
    print(resume.text)

    #DATE
    date = soup.find("span", {"class" : "publication_date"}).text
    print(date)

    #TAGS
    tags_liste = soup.find_all("ul", {"class": "list-inline keywords"})
    tags = []
    for tag in tags_liste:
        a = tag.find_all("a")
        for tag in a:
            if re.search("tag", tag["href"]) != None:
                tags += tag.contents
    print(tags)

    #AUTHORS
    if type == "Interview":
        liste = soup.find_all("ul", {"class": "list-inline keywords"})
        authors = []
        for elem in liste:
            a = elem.find_all("a")
            for author in a:
                if re.search("auteurs", author["href"]) != None:
                    authors.append(author.contents[0].strip())
    else:
        auth = soup.find_all("span", {"class" : "author-name"})
        authors = []
        for author in auth:
            author = repr(author.text).replace("\\xa0", " ").replace("\\t", "").replace("\\n", "").replace("'", "")
            if author not in authors:
                authors.append(author)
    print(authors)

    #ARTICLE
    if type != "Interview":
        article = soup.find("div",  {"class" : "ezrichtext-field"}).text
    else:
        lines = soup.find_all("div", {"class" : "interview-item"})
        article = ""
        for i, line in enumerate(lines):
            article += line.find("p", {"class" : "over-under-line"}).text + " "
            temp = line.find_all("div", {"class" : "ezrichtext-field"})
            for info in temp:
                article += info.find("p").text
    print(article)
        
    #Liens
    page_links = soup.find_all("h3", {"class" : "h3 uppercase"})
    links = []
    for link in page_links:
        links.append("https://www.millenaire3.com"+link.a["href"])
    print(links)


In [7]:
for url in urls:
    scrap_page(url)

https://www.millenaire3.com/ressources/2023/changer-les-comportements-revue-et-enseignements-des-actions-metropolitaines
Changer les comportements : revue et enseignements des actions métropolitaines
Étude
La transition écologique nécessite d’opérer des changements profonds et rapides à toutes les échelles de la société. Sa réalisation demande de faire évoluer les manières d’agir, les choix de vie individuels et collectifs et les infrastructures techniques ; autrement dit, d’agir sur les modes de vie.
Un mode de vie est plus qu’une somme de comportements, et agir sur eux nécessite de repenser les leviers d’action. Cependant, cela ne veut pas dire que la volonté de changer les comportements devient inopérante : elle a son utilité chez certains publics et elle est efficace pour modifier certaines pratiques, comme le geste de tri par exemple. 
Partant du constat que la Métropole de Lyon a déjà une expérience sur l’action en faveur d’un changement de comportements et de modes de vie, ce do

In [366]:
html = requests.get(url4).text
soup = BeautifulSoup(html, "html.parser")

In [375]:
soup.body

<body id="millenaire-3" itemscope="" itemtype="http://schema.org/WebPage">
<header id="header">
<div class="container" id="print-top-menu">
<div class="row">
<div class="col-xs-60">
<img alt="" src="/assets/images/logo-millenaire-3-mini-blanc.png" style="float: left;"/>
<img alt="" src="/assets/images/logo-grand-lyon-la-metropole-blanc.png" style="float: right;"/>
</div>
</div>
</div>
<div id="top-menu">
<div class="container">
<div class="row">
<div class="col-xs-60">
<div class="clearfix" id="top-menu-content">
<a class="link-sprite-focus m3-icons-btn-toggle-menu" href="#" id="btn-toggle-menu" title="Accéder au menu"></a>
<a href="/" id="link-logo-millenaire-3-mini" title="Retourner à l'accueil de Millénaire 3">
<img alt="" class="img-responsive" id="logo-mini-m3" src="/assets/images/logo-millenaire-3-mini.png">
</img></a>
<a class="skippy sr-only sr-only-focusable" href="#content" title=""><span class="skiplink-text">Aller au contenu </span></a>
<a class="skippy sr-only sr-only-focu

In [401]:
lines = soup.find_all("div", {"class" : "interview-item"})
article = ""
for i, line in enumerate(lines):
    article += line.find("p", {"class" : "over-under-line"}).text+" "
    temp = line.find_all("div", {"class" : "ezrichtext-field"})
    for info in temp:
        article += info.find("p").text
print(article)

Pouvez-vous présenter en quelques mots l’Institut Transitions ? L’Institut Transitions est un organisme de formation né, en septembre 2019, de deux constats. Tout d’abord, de plus en plus de personnes souhaitent s’engager dans leur vie professionnelle et participer à la transition écologique et solidaire, mais elles ne savent pas comment s’y prendre : où se former ? Qui rencontrer ? Quelles compétences acquérir ? Comment se construire un réseau ? Second constat : les structures porteuses de la transition ont besoin de forces vives pour grandir et avoir plus d’impacts. Association non lucrative, l’Institut répond à ce double besoin : accompagner les personnes dans leur évolution professionnelle, et les structures face aux défis qu’elles rencontrent.Combien êtes-vous ? Notre équipe regroupe aujourd’hui 3 permanentes structurelles, une vingtaine de formateurs et d’accompagnateurs engagés presque au quotidien et plus de 100 partenaires de la région lyonnaise.Comment vous est venue l’idée d